In [79]:
# %load data_parser.py
import os
import json
import numpy as np


def get_data(file_dir):
    """
    Will read JSON files from the provided file directory and subdirectories and return a list of dicts
    :param file_dir: directory where to look for input JSON files
    :return: list of dicts converted from JSON files
    """
    print('Starting to read JAAD json files!')
    json_list = []

    for dirpath, dirnames, filenames in os.walk(file_dir):
        JAAD_list = [pos_json for pos_json in filenames if pos_json.endswith('.json')]
        for js in JAAD_list:
            with open(os.path.join(dirpath, js)) as json_file:
                json_list.append(json.load(json_file))

    return json_list


def get_data_TCG(file_dir: str) -> (np.ndarray, dict):
    """
    Read in the TCG dataset and its annotation
    :param file_dir: directory where data and annotations from TCG are stored
    :return: Data as np array and annotations as dict
    """
#     tcg_data = np.load(file_dir + 'tcg_data.npy', allow_pickle=True)
    tcg_data = np.load('tcg_data.npy', allow_pickle=True)
#     with open(file_dir + 'tcg.json') as json_file:
    with open('tcg.json') as json_file:
        tcg_json = json.load(json_file)

    return tcg_data, tcg_json


# get_data('../data/JAAD_output_JSON')

# example usage
tcg_d, tcg_j = get_data_TCG('../data/TCG/')
print(len(tcg_d))
print(tcg_j.keys())
print(len(tcg_j['sequences']))
print(tcg_d[0])
print(tcg_d[0].shape)

550
dict_keys(['sequences', 'dataset', 'description', 'version', 'joint_dictionary'])
550
[[[-1.30057383e-01 -2.87529844e+00  3.21951370e+01]
  [ 1.01548117e-01 -8.05008894e-01  5.99461899e+01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  ...
  [ 9.12524309e+00 -1.62379413e+00 -5.10988693e+01]
  [ 8.91745327e+00  1.96904989e-01 -6.25629425e+00]
  [-4.10138145e-02 -1.42855572e+00  1.86052094e+01]]

 [[-1.28102608e-01 -2.87761379e+00  3.21949310e+01]
  [ 1.02079381e-01 -8.08472654e-01  5.99459534e+01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  ...
  [ 9.12010052e+00 -1.62173269e+00 -5.10997581e+01]
  [ 8.91693451e+00  1.96307000e-01 -6.25704956e+00]
  [-3.96665519e-02 -1.42961226e+00  1.86051254e+01]]

 [[-1.27165717e-01 -2.88150805e+00  3.21945801e+01]
  [ 1.02370057e-01 -8.12958233e-01  5.99454956e+01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  ...
  [ 9.12307961e+00 -1.62498423e+00 -5.10996361e+01]
  [ 8.91685468e+00  1.97516109e-01 -6.25712585e+00]
  [-

In [80]:
tcg_j['sequences'][549]

{'subject_id': 5,
 'junction': 'X',
 'scene_id': 5,
 'scene_agents': [{'id': 1,
   'position': 'right',
   'intention': 'goes left',
   'queue': 3},
  {'id': 4, 'position': 'bottom', 'intention': 'goes straight', 'queue': 1},
  {'id': 3, 'position': 'left', 'intention': 'goes left', 'queue': 3},
  {'id': 2, 'position': 'top', 'intention': 'goes left', 'queue': 2}],
 'agent_number': 2,
 'annotation': [['inactive', 'normal-pose', 0, 717],
  ['inactive', 'transition', 717, 741],
  ['stop', 'right-static', 741, 894],
  ['inactive', 'transition', 894, 906],
  ['inactive', 'normal-pose', 906, 3574],
  ['inactive', 'transition', 3574, 3614],
  ['go', 'right-dynamic', 3614, 3795],
  ['inactive', 'normal-pose', 3795, 5659],
  ['inactive', 'out-of-vocabulary', 5659, 5921],
  ['inactive', 'normal-pose', 5921, 6132]],
 'num_frames': 6132}

In [81]:
tcg_d.shape

(550,)

In [98]:
data=tcg_d[0]
a=0
for i in range(550):
    a+=tcg_j['sequences'][i]['num_frames']
    data=np.concatenate((data,tcg_d[i]),axis=0)

In [99]:
data=data[3400:]

In [100]:
data.shape

(1493472, 17, 3)

In [85]:
a

1493472

## data length:1493472

##### set 500 sequences as training data,50 sequences as test data

In [104]:
length_tr=0
for i in range(500):
    length_tr+=tcg_j['sequences'][i]['num_frames']
length_tr

1352138

## Turn 3D data to Tensor

In [101]:
import tensorflow as tf

In [102]:
tensor = tf.convert_to_tensor(data)

In [103]:
tensor.shape

TensorShape([1493472, 17, 3])

In [91]:
lst=tcg_j['sequences'][0]['annotation']

In [92]:
for i in range(1,550):
    lst+=tcg_j['sequences'][i]['annotation']

In [93]:
lst[:10]

[['inactive', 'normal-pose', 0, 1045],
 ['inactive', 'transition', 1045, 1112],
 ['stop', 'both-static', 1112, 1461],
 ['inactive', 'transition', 1461, 1573],
 ['clear', 'right-static', 1573, 1818],
 ['inactive', 'transition', 1818, 1951],
 ['go', 'both-static', 1951, 2482],
 ['inactive', 'transition', 2482, 2582],
 ['inactive', 'normal-pose', 2582, 3400],
 ['inactive', 'normal-pose', 0, 735]]

In [107]:
# inactive=0
# stop=1
# clear=2
# go=3

#create labels(y)
y_all=[]
for item in lst:
    if item[0]=='inactive':
        y_all.append('0'*(item[3]-item[2]))
    elif item[0]=='stop':
        y_all.append('1'*(item[3]-item[2]))
    elif item[0]=='clear':
        y_all.append('2'*(item[3]-item[2]))
    elif item[0]=='go':
        y_all.append('3'*(item[3]-item[2]))

alle=''
for i in y_all:
    alle+=i

lst_new=[int(char) for char in alle]
label=np.array(lst_new)
label.shape

(1493472,)

In [108]:
label[0]

0

## split train and test data

In [109]:
transposed_tensor = tf.transpose(tensor, perm=[1, 0, 2])

In [111]:
reshaped_data = tf.transpose(tensor,perm=[0,2,1])
reshaped_data = tf.reshape(reshaped_data,shape=(-1,1493472,17))
reshaped_data

<tf.Tensor: shape=(3, 1493472, 17), dtype=float64, numpy=
array([[[-1.30057383e-01,  1.01548117e-01,  0.00000000e+00, ...,
          9.12524309e+00,  8.91745327e+00, -4.10138145e-02],
        [-2.87529844e+00, -8.05008894e-01,  0.00000000e+00, ...,
         -1.62379413e+00,  1.96904989e-01, -1.42855572e+00],
        [ 3.21951370e+01,  5.99461899e+01,  0.00000000e+00, ...,
         -5.10988693e+01, -6.25629425e+00,  1.86052094e+01],
        ...,
        [ 1.11139622e+00,  3.32041788e+00,  0.00000000e+00, ...,
         -1.04771824e+00, -8.81053474e+00,  7.35899559e-01],
        [ 1.55280398e+00,  3.57639917e-02,  0.00000000e+00, ...,
         -1.50032096e+01, -2.61914082e+00,  5.63636027e-01],
        [ 3.22618408e+01,  6.00113983e+01,  0.00000000e+00, ...,
         -4.82827454e+01, -5.84941101e+00,  1.86358490e+01]],

       [[ 1.16337247e+00,  3.44552098e+00,  0.00000000e+00, ...,
         -1.07100658e+00, -8.80291455e+00,  7.64644807e-01],
        [ 1.54434521e+00,  6.63436904e-02,  0

In [112]:
mean,var=tf.nn.moments(tensor,axes=[0,1,2])
normalized_tensor=(tensor-mean)/tf.sqrt(var)

In [113]:
normalized_tensor

<tf.Tensor: shape=(1493472, 17, 3), dtype=float64, numpy=
array([[[-0.02455627, -0.11752077,  1.0701    ],
        [-0.01671322, -0.04741275,  2.00985813],
        [-0.02015203, -0.02015203, -0.02015203],
        ...,
        [ 0.28886404, -0.07513999, -1.75055765],
        [ 0.28182748, -0.01348406, -0.23201438],
        [-0.02154091, -0.06852846,  0.60989242]],

       [[-0.02449008, -0.11759918,  1.07009303],
        [-0.01669522, -0.04753004,  2.00985012],
        [-0.02015203, -0.02015203, -0.02015203],
        ...,
        [ 0.2886899 , -0.07507018, -1.75058775],
        [ 0.28180991, -0.01350431, -0.23203996],
        [-0.02149529, -0.06856424,  0.60988958]],

       [[-0.02445835, -0.11773105,  1.07008114],
        [-0.01668538, -0.04768194,  2.00983462],
        [-0.02015203, -0.02015203, -0.02015203],
        ...,
        [ 0.28879078, -0.07518029, -1.75058361],
        [ 0.28180721, -0.01346337, -0.23204254],
        [-0.02148245, -0.06863847,  0.60988389]],

       ...,

  

In [114]:
x_tr=normalized_tensor[:1352138]
x_te=normalized_tensor[1352138:]
y_tr=label[:1352138]
y_te=label[1352138:]

In [115]:
x_tr.shape

TensorShape([1352138, 17, 3])

### y --> onehot coding

In [116]:
# uniq=np.unique(label)
# sorted_v=np.sort(uniq)
# one_hot=np.zeros((len(label),len(sorted_v)))
# for i,v in enumerate(label):
#     index=np.where(sorted_v==v)[0][0]
#     one_hot[i,index]=1
# # print(one_hot)
# y_tr=one_hot[:2482]
# y_te=one_hot[2482:]

## LSTM 

In [117]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input

# define model
timesteps=1352138
features=17
coordinate_values=3
model = Sequential()
# model.add(LSTM(units=32, input_shape=(timesteps, features))) 
model.add(LSTM(64,input_shape=(features, coordinate_values)))
model.add(Dense(1))  # output layer，units is the unit number for output

# compile model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

# train model
model.fit(x_tr, y_tr, epochs=10, batch_size=32) 

# prediction
predictions = model.predict(x_te)




Epoch 1/10
42255/42255 [==============================] - 297s 7ms/step - loss: 0.4224 - accuracy: 0.7057
Epoch 2/10
42255/42255 [==============================] - 316s 7ms/step - loss: 0.2364 - accuracy: 0.7443
Epoch 3/10
42255/42255 [==============================] - 291s 7ms/step - loss: 0.1682 - accuracy: 0.7577
Epoch 4/10
42255/42255 [==============================] - 365s 9ms/step - loss: 0.1348 - accuracy: 0.7652
Epoch 5/10
42255/42255 [==============================] - 305s 7ms/step - loss: 0.1167 - accuracy: 0.7696
Epoch 6/10
42255/42255 [==============================] - 309s 7ms/step - loss: 0.1055 - accuracy: 0.7722
Epoch 7/10
42255/42255 [==============================] - 299s 7ms/step - loss: 0.0982 - accuracy: 0.7742
Epoch 8/10
42255/42255 [==============================] - 314s 7ms/step - loss: 0.0933 - accuracy: 0.7753
Epoch 9/10
42255/42255 [==============================] - 314s 7ms/step - loss: 0.0895 - accuracy: 0.7760
Epoch 10/10
42255/42255 [=====================

In [118]:
model.evaluate(y_te,predictions)

ValueError: in user code:

    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\keras\engine\training.py:1323 test_function  *
        return step_function(self, iterator)
    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\keras\engine\training.py:1314 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\Miniforge\envs\Ming\lib\site-packages\tensorflow\python\keras\engine\training.py:1307 run_step  **
        out

In [ ]:
y_tr,len(y_tr)

In [ ]:
# predictions
# len(predictions)

In [ ]:
round_pred=np.round(predictions).astype(int)

In [ ]:
print(round_pred[-10:])